In [4]:
import pandas as pd
# Open the .csv.gz file
df = pd.read_csv('../data/01-raw_data/raw_data.csv.gz')

# Display the first few rows
print(df.head())

   YEAR  SAMPLE  SERIAL       CBSERIAL  HHWT        CLUSTER  STRATA  GQ  \
0  2022  202201       1  2022010000031  69.0  2022000000011  280301   3   
1  2022  202201       2  2022010000111  22.0  2022000000021  200001   3   
2  2022  202201       3  2022010000200  45.0  2022000000031  280301   3   
3  2022  202201       4  2022010000261   4.0  2022000000041  110001   4   
4  2022  202201       5  2022010000296  47.0  2022000000051  150201   3   

   PERNUM  PERWT  SEX  AGE  MARST  EDUC  EDUCD  SCHLTYPE  OCC2010  INCTOT  
0       1   69.0    2   85      5     7     71         1     9920   18800  
1       1   22.0    1   51      5     6     64         1     5620   12500  
2       1   45.0    2   36      6     2     26         1     8800   16400  
3       1    4.0    1   74      6     0      2         1     9920    8600  
4       1   47.0    1   49      4     7     71         1     6230    5000  
